In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
from scipy import spatial
from skimage.io import imread_collection, imread, imshow
import sys
from sklearn.decomposition import PCA, TruncatedSVD
from PIL import Image
import random
from scipy.spatial import distance
sys.setrecursionlimit(100000)

In [2]:
def load(redux, redux_type):
    #PROCESAMIENTO DE DATOS
    borrarcomillas = lambda x: x.replace('"', '')
    names = np.genfromtxt('clase.csv', dtype=None, delimiter=",", skip_header=1, usecols=1, encoding='utf-8', converters={1: borrarcomillas})
    data = np.genfromtxt('dataset.csv', delimiter=",")

    #REDUCCION DE DIMENSIONALIDAD
    svd = TruncatedSVD(n_components=redux)
    pca = PCA(n_components=redux)
    datos = data
    if redux_type == 0:
        datos = svd.fit(data).transform(data)
    elif redux_type == 1:
        datos = pca.fit(data).transform(data)

    #VARIABLES UTILES
    n_datos = len(datos)
    dim_datos = len(datos[0])
    
    #CARGA DE PUNTOS
    puntos = []
    for i in range(n_datos):
        puntos.append(np.array(datos[i]))
  
    #RETORNO DE LOS PUNTOS CARGADOS, LA DIMENSION DE LOS DATOS Y LOS LABELS CORRECTOS
    return puntos, dim_datos, names

In [3]:
def dbscan(idx, punto, clusterId, radio):
	global visitados
	global clusteres
	if clusteres.get(clusterId) == None:
		clusteres[clusterId] = []
	if visitados[idx]:
		return
	if not visitados[idx]: # si no ha sido visitado
		vecinos = kdtree.query_ball_point(punto, radio) # agarro los puntos dentro del radio
		visitados[idx] = True # marco el centro como visitado
		visitados_en_rango = 0 # cuantos de mis vecinos han sido visitados
		no_visitados = [] # vecinos que no han sido visitados
		no_visitados_indices = [] # indices de los vecinos que no han sido visitados
		for vecino in vecinos: # por cada vecino
			if visitados[vecino]: # si ha sido visitado
				visitados_en_rango += 1 # aumentar contador de ya visitados
			else: # si no ha sido visitado
				clusteres[clusterId].append(names[vecino])
				no_visitados.append(puntos[vecino]) # añadir el vecino a la lista de vecinos no visitados
				no_visitados_indices.append(vecino)
		if visitados_en_rango == len(vecinos) : # si todos mis vecinos han sido visitados, ya termine con el cluster
			return
		for indice, no_visitado in enumerate(no_visitados):
			if indice >= len(no_visitados_indices):
				continue
			dbscan(idx=no_visitados_indices[indice], punto=no_visitado, clusterId=clusterId, radio=radio)

In [4]:
def mean_distance():
    mean_distance = 0
    for i in range(len(puntos)):
        for j in range(len(puntos)):
            if i != j:
                mean_distance += distance.euclidean(puntos[i], puntos[j])
    mean_distance = mean_distance/len(puntos)**2
    return mean_distance

In [5]:
def find_no_visitado():
	for idx, punto in enumerate(visitados):
		if not punto:
			return [idx, puntos[idx]]
	return []

In [6]:
redux = 100
redux_type = 0

puntos, dim_datos, names = load(redux, redux_type)
visitados = [False] * len(puntos)
kdtree = spatial.KDTree(puntos)

In [7]:
clusteres = {}
visitados = [False] * len(puntos)

pt = find_no_visitado()
clusterId = 0
min_points = 20
radio = mean_distance() - 55

while pt != []:
    dbscan(pt[0], pt[1], clusterId, radio)
    clusterId += 1
    pt = find_no_visitado()

In [8]:
final_clusteres = {}
e = 10

for key in clusteres.keys():
    if len(clusteres[key]) == 0 or len(clusteres[key]) < min_points:
        continue
    cur_cluster = {}
    for label in clusteres[key]:
        if cur_cluster.get(label) == None:
            cur_cluster[label] = 1
        else:
            cur_cluster[label] = cur_cluster[label] + 1
    max_repeticion = 0
    final_label = ''
    for key_ in cur_cluster.keys():
        if max_repeticion < cur_cluster[key_]:
            max_repeticion = cur_cluster[key_]
            final_label = key_ + str(e) # Para evitar colisiones
    e += 1
    final_clusteres[final_label] = clusteres[key]

In [9]:
for key in final_clusteres.keys():
    count = 0
    size = len(key)

    for label in final_clusteres[key]:
        if label == key[:size - 2]:
            count += 1
    
    print("Cluster:", key)
    print(round(count/len(final_clusteres[key]) * 100, 4), "% de valores correctos.", "(size: " + str(len(final_clusteres[key])) + ")\n")

Cluster: kidney10
100.0 % de valores correctos. (size: 174)

Cluster: cerebellum11
53.3981 % de valores correctos. (size: 824)

Cluster: colon12
100.0 % de valores correctos. (size: 436)

Cluster: liver13
100.0 % de valores correctos. (size: 185)

Cluster: endometrium14
100.0 % de valores correctos. (size: 61)

